In [1]:
import numpy as np
import sympy as sp
from typing import List, Tuple, Dict, Any, Set
from scipy.stats import levy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

class all_parameter_generation:
    """
    Generate state transitions and random parameters (a, b, c, enzyme) for an n-site phosphorylation model.

    Args:
        n: number of sites (int)
        distribution: distribution name ("gamma" supported)
        params: parameters for the distribution (for gamma: [shape, scale])
        verbose: if True, prints transitions and matrices
    """
    def __init__(self, n: int, reaction_types: str, distribution: str, distribution_paramaters: List[float], verbose: bool = False):
        self.n = n
        self.num_states = 2 ** n
        self.distribution = distribution
        self.params = distribution_paramaters
        self.reaction_types = reaction_types
        self.verbose = verbose
        self.rng = np.random.default_rng()
        
    @staticmethod
    def padded_binary(i: int, n: int) -> str:
        return bin(i)[2:].zfill(n)

    @staticmethod
    def binary_string_to_array(string: str) -> np.ndarray:
        return np.array([int(i) for i in string], dtype=int)

    def calculate_valid_transitions(self) -> Tuple[List[List[Any]], List[List[Any]]]:
        """
        Returns:
            valid_X_reactions: list of [state_i_str, state_j_str, i, j, "E"]
            valid_Y_reactions: list of [state_i_str, state_j_str, i, j, "F"]
        """
        all_states = [self.padded_binary(i, self.n) for i in range(self.num_states)]

        valid_difference_vectors: Set[Tuple[int, ...]] = set()
        valid_X_reactions: List[List[Any]] = []
        valid_Y_reactions: List[List[Any]] = []

        for i in range(self.num_states):
            arr_i = self.binary_string_to_array(all_states[i])
            for j in range(self.num_states):
                if i == j:
                    continue
                arr_j = self.binary_string_to_array(all_states[j])
                diff = arr_j - arr_i
                # if self.reaction_types == "distributive":
                    
                hamming_weight = np.sum(np.abs(diff))

                if hamming_weight == 1:
                    # +1 -> phosphorylation (E), -1 -> dephosphorylation (F)
                    element = "E" if np.any(diff == 1) else "F"
                    if element == "E":
                        if self.verbose:
                            print(f"{all_states[i]} --> {all_states[j]} (E), {i}, {j}")
                        valid_X_reactions.append([all_states[i], all_states[j], i, j, element])
                    else:
                        if self.verbose:
                            print(f"{all_states[i]} --> {all_states[j]} (F), {i}, {j}")
                        valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])
                    valid_difference_vectors.add(tuple(diff))

        return valid_X_reactions, valid_Y_reactions
    
    def alpha_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray,
                                            Dict[int, List[int]], Dict[int, List[int]],
                                            Dict[int, List[int]], Dict[int, List[int]]]:
        
        valid_X_reactions, valid_Y_reactions = self.calculate_valid_transitions()

        shape, scale = self.params

        alpha_matrix = np.zeros((self.num_states, self.num_states))

        for _, _, i, j, _ in valid_X_reactions:

            alpha_matrix[i][j] = self.rng.gamma(shape, scale)

        return alpha_matrix

    def beta_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray,
                                            Dict[int, List[int]], Dict[int, List[int]],
                                            Dict[int, List[int]], Dict[int, List[int]]]:
        
        valid_X_reactions, valid_Y_reactions = self.calculate_valid_transitions()

        shape, scale = self.params
        beta_matrix = np.zeros((self.num_states, self.num_states))
        
        for _, _, i, j, _ in valid_Y_reactions:

            beta_matrix[i][j] = self.rng.gamma(shape, scale)

        return beta_matrix
    
    def k_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for a_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            k_positive_rates = self.rng.gamma(shape, scale, self.num_states - 1)
            k_negative_rates = self.rng.gamma(shape, scale, self.num_states - 1)
        if self.distribution == "levy":
            k_positive_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
            k_negative_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
        # k_positive_rates[-1] = 0
        # k_negative_rates[-1] = 0
        
        return k_positive_rates, k_negative_rates

    def p_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for b_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            p_positive_rates = self.rng.gamma(shape, scale, self.num_states - 1)
            p_negative_rates = self.rng.gamma(shape, scale, self.num_states - 1)
        if self.distribution == "levy":
            p_positive_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
            p_negative_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
        # p_positive_rates[0] = 0
        # p_negative_rates[0] = 0

        return p_positive_rates, p_negative_rates
    

In [2]:
from sympy import solve
# from sympy.abc import A0, A1, B0, C1, X, Y
X = sp.Symbol('X'); Y = sp.Symbol('Y')
A = sp.IndexedBase('A')
B = sp.IndexedBase('B')
C = sp.IndexedBase('C')
solutions = solve([-X*A[0] + B[0] + C[1], -Y*A[1] + C[1] + B[0], X*A[0] - 2*B[0], Y*A[1] - 2*C[1], -X*A[0] + 2*B[0], -Y*A[1] + 2*C[1]], [A[0], A[1], B[0], C[1], X, Y], dict=True)
solutions[0][A[0]]


Y*A[1]/X

In [3]:
solutions[0][B[0]]

Y*A[1]/2

In [4]:
solutions[0][C[1]]

Y*A[1]/2

In [5]:
import sympy as sp

def build_phos_odes(n, prefix=""):
    """
    Build symbolic ODEs for the phosphorylation system for given n (N = 2**n).
    Returns a dict with keys:
      N, a, b, c, x, y,
      Kp, Km, Pp, Pm, Alpha, Beta, DAlpha, DBeta,
      adot, bdot, cdot, xdot, ydot,
      ones (vector of ones)
    """
    N = 2**n
    a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])
    b_syms = sp.symbols([f"{prefix}b_{i}" for i in range(N)])
    c_syms = sp.symbols([f"{prefix}c_{i}" for i in range(N)])
    x_sym, y_sym = sp.symbols(f"{prefix}x {prefix}y")

    a = sp.Matrix(N, 1, lambda i,j: a_syms[i])
    b = sp.Matrix(N, 1, lambda i,j: b_syms[i])
    b[-1] = 0
    print(b)
    c = sp.Matrix(N, 1, lambda i,j: c_syms[i])
    c[0] = 0

    print(c)

    x = x_sym
    y = y_sym

    kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N)])
    kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N)])
    pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(N)])
    pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(N)])

    Kp = sp.diag(*kplus)
    Km = sp.diag(*kminus)
    Pp = sp.diag(*pplus)
    Pm = sp.diag(*pminus)

    valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n)

    # compute allowed transitions
    valid_X, valid_Y = calculate_valid_transitions(n)
    
    # Build Alpha and Beta with symbols only at allowed (j,k)
    Alpha = sp.zeros(N, N)
    Beta  = sp.zeros(N, N)
    # create symbols for allowed entries
    for (_, _, j,k, _) in valid_X:
        Alpha[j,k] = sp.symbols(f"{prefix}alpha_{j}_{k}")
    for (_, _, j,k, _) in valid_Y:
        Beta[j,k] = sp.symbols(f"{prefix}beta_{j}_{k}")
      
    # Set last column and row of Alpha to zero (no transitions from or to fully phosphorylated state)
    # Alpha[:, N-1] = sp.zeros(N, 1)
    # Alpha[N-1, :] = sp.zeros(1, N)
    # # Set first column and row of Beta to zero (no transitions from or to fully de
    # Beta[:, 0] = sp.zeros(N, 1)
    # Beta[0, :] = sp.zeros(1, N)
    print("Alpha matrix:")
    sp.pprint(Alpha)
    print("Beta matrix:")
    sp.pprint(Beta)
    ones = sp.Matrix([1]*N)
    Alpha_row_sums = Alpha * ones
    Beta_row_sums = Beta * ones
    DAlpha = sp.diag(*[Alpha_row_sums[i,0] for i in range(N)])
    DBeta  = sp.diag(*[Beta_row_sums[i,0]  for i in range(N)])

    # a_adjusted_for_b_dot = a.copy(); a_adjusted_for_b_dot[-1] = 0
    # a_adjusted_for_c_dot = a.copy(); a_adjusted_for_c_dot[0] = 0

    # a_adjusted_for_x_dot = a.copy(); a_adjusted_for_x_dot[-1] = 0
    # a_adjusted_for_y_dot = a.copy(); a_adjusted_for_y_dot[0] = 0

    adot = Km * b + Pm * c + Alpha.T * b + Beta.T * c - x * (Kp * a) - y * (Pp * a)
    bdot = x * (Kp * a) - Km * b - DAlpha * b
    cdot = y * (Pp * a) - Pm * c - DBeta * c

    kplus_vec = sp.Matrix(kplus)
    pplus_vec = sp.Matrix(pplus)

    xdot = - x * (kplus_vec.T * a)[0] + (ones.T * (Km + DAlpha) * b)[0]
    ydot = - y * (pplus_vec.T * a)[0] + (ones.T * (Pm + DBeta) * c)[0]
    
    return {
        "N": N,
        "a": a, "b": b, "c": c, "x": x, "y": y,
        "Kp": Kp, "Km": Km, "Pp": Pp, "Pm": Pm,
        "Alpha": Alpha, "Beta": Beta,
        "DAlpha": DAlpha, "DBeta": DBeta,
        "ones": ones,
        "adot": sp.simplify(adot), "bdot": sp.simplify(bdot), "cdot": sp.simplify(cdot),
        "xdot": sp.simplify(xdot), "ydot": sp.simplify(ydot)
    }

# Example: build symbolic ODEs for n=2 (N=4)
n = 1
odes = build_phos_odes(n)
odes["adot"][0]
# sp.pprint(odes["adot"])
# sp.pprint(odes["bdot"])
# sp.pprint(odes["cdot"])
# sp.pprint(odes["xdot"])
# sp.pprint(odes["ydot"])


Matrix([[b_0], [0]])
Matrix([[0], [c_1]])


NameError: name 'calculate_valid_transitions' is not defined

In [6]:
odes["bdot"][0]


NameError: name 'odes' is not defined

In [7]:
odes["bdot"][1]


NameError: name 'odes' is not defined

In [8]:
odes["cdot"][1]

NameError: name 'odes' is not defined

In [9]:
odes["xdot"]

NameError: name 'odes' is not defined

In [10]:
odes["ydot"]

NameError: name 'odes' is not defined

# BEST SYMBOLIC ODE GENERATOR:

In [11]:
import sympy as sp
def padded_binary(i: int, n: int) -> str:
    return bin(i)[2:].zfill(n)

# Replace this method in your class
@staticmethod
def binary_string_to_array(string: str) -> np.ndarray:
    # use a comprehension so we don't rely on the builtin `list` name being callable
    return np.array([int(ch) for ch in string], dtype=int)

def calculate_valid_transitions(n: int):
    num_states = 2**n
    all_states = [padded_binary(i, n) for i in range(num_states)]
    
    # print(f"Total number of states: {num_states}")
    # print("Valid single-step transitions:")

    valid_difference_vectors = set()
    
    valid_X_reactions = [] # distributively
    valid_Y_reactions = [] # distributively

    for i in range(num_states):
        for j in range(num_states):
            # Do not consider transitions from a state to itself
            if i == j:
                continue

            if np.sum(np.abs(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]))) == 1:
                # Determine if it's a phosphorylation or dephosphorylation event
                # A +1 indicates phosphorylation, a -1 indicates dephosphorylation
                element = "X" if np.any(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]) == 1) else "Y"
                
                if element == "X":
                    print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                    valid_X_reactions.append([all_states[i], all_states[j], i, j, element])
                if element == "Y":
                    print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                    valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])

                valid_difference_vectors.add(tuple(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i])))

    return valid_X_reactions, valid_Y_reactions

def build_phos_odes(n, prefix=""):
    """
    Build symbolic ODEs for the phosphorylation system for given n (N = 2**n).
    Returns a dict with keys:
      adot, bdot, cdot, xdot, ydot,
    """
    N = 2**n
    a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])
    b_syms = sp.symbols([f"{prefix}b_{i}" for i in range(N)])
    c_syms = sp.symbols([f"{prefix}c_{i}" for i in range(N)])
    x_sym, y_sym = sp.symbols(f"{prefix}x {prefix}y")

    a = sp.Matrix(N, 1, lambda i,j: a_syms[i])
    b = sp.Matrix(N, 1, lambda i,j: b_syms[i])
    c = sp.Matrix(N, 1, lambda i,j: c_syms[i])
    
    x = x_sym
    y = y_sym

    kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N)]); kplus[-1] = sp.Integer(0)
    kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N)]); kminus[-1] = sp.Integer(0)
    pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(N)]); pplus[0] = sp.Integer(0)
    pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(N)]); pminus[0] = sp.Integer(0)

    Kp = sp.diag(*kplus)
    Km = sp.diag(*kminus)
    Pp = sp.diag(*pplus)
    Pm = sp.diag(*pminus)

    # compute allowed transitions
    valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n)
    
    # Build Alpha and Beta with symbols only at allowed (j,k)
    Alpha = sp.zeros(N, N)
    Beta  = sp.zeros(N, N)
    # create symbols for allowed entries
    for (_, _, j,k, _) in valid_X_reactions:
        Alpha[j,k] = sp.symbols(f"{prefix}alpha_{j}_{k}")
    for (_, _, j,k, _) in valid_Y_reactions:
        Beta[j,k] = sp.symbols(f"{prefix}beta_{j}_{k}")

    ones = sp.Matrix([1]*N)
    Alpha_row_sums = Alpha * ones
    Beta_row_sums = Beta * ones
    DAlpha = sp.diag(*[Alpha_row_sums[i, 0] for i in range(N)])
    DBeta  = sp.diag(*[Beta_row_sums[i, 0] for i in range(N)])
    print(DAlpha)
    print(DBeta)
    adot = Km * b + Pm * c + Alpha.T * b + Beta.T * c - x * (Kp * a) - y * (Pp * a)
    bdot = x * (Kp * a) - Km * b - DAlpha * b
    cdot = y * (Pp * a) - Pm * c - DBeta * c
    bdot[-1] = sp.Integer(0)
    cdot[0] = sp.Integer(0)
    kplus_vec = sp.Matrix(kplus)
    pplus_vec = sp.Matrix(pplus)

    xdot = - x * (kplus_vec.T * a)[0] + (ones.T * (Km + DAlpha) * b)[0]
    ydot = - y * (pplus_vec.T * a)[0] + (ones.T * (Pm + DBeta) * c)[0]
    # print(type(adot[0]))

    return {
        "adot": sp.expand(adot), "bdot": sp.expand(bdot), "cdot": sp.expand(cdot),
        "xdot": sp.expand(xdot), "ydot": sp.expand(ydot),
    }

# Example: build symbolic ODEs for n=2 (N=4)
n = 2
odes = build_phos_odes(n)
odes["cdot"][1]
# sp.pprint(odes["adot"])
# sp.pprint(odes["bdot"])
# sp.pprint(odes["cdot"])
# sp.pprint(odes["xdot"])
# sp.pprint(odes["ydot"])

00 --> 01 (X), 0 -> 1
00 --> 10 (X), 0 -> 2
01 --> 00 (Y), 1 -> 0
01 --> 11 (X), 1 -> 3
10 --> 00 (Y), 2 -> 0
10 --> 11 (X), 2 -> 3
11 --> 01 (Y), 3 -> 1
11 --> 10 (Y), 3 -> 2
Matrix([[alpha_0_1 + alpha_0_2, 0, 0, 0], [0, alpha_1_3, 0, 0], [0, 0, alpha_2_3, 0], [0, 0, 0, 0]])
Matrix([[0, 0, 0, 0], [0, beta_1_0, 0, 0], [0, 0, beta_2_0, 0], [0, 0, 0, beta_3_1 + beta_3_2]])


a_1*p^+_1*y - beta_1_0*c_1 - c_1*p^-_1

In [12]:
odes["cdot"][2]


a_2*p^+_2*y - beta_2_0*c_2 - c_2*p^-_2

In [13]:
odes["cdot"][3]


a_3*p^+_3*y - beta_3_1*c_3 - beta_3_2*c_3 - c_3*p^-_3

In [14]:
odes["xdot"]


-a_0*k^+_0*x - a_1*k^+_1*x - a_2*k^+_2*x + alpha_0_1*b_0 + alpha_0_2*b_0 + alpha_1_3*b_1 + alpha_2_3*b_2 + b_0*k^-_0 + b_1*k^-_1 + b_2*k^-_2

# PURELY SYMBOLIC:

In [15]:
prefix=""
n = 1
N = 2**n
a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])

# gen = all_parameter_generation(n, "distributive", "gamma", (1, 2), verbose = False)
# alpha_matrix = gen.alpha_parameter_generation()
# beta_matrix = gen.beta_parameter_generation()
# k_positive_rates, k_negative_rates = gen.k_parameter_generation()
# p_positive_rates, p_negative_rates = gen.p_parameter_generation()
# k_positive_rates = np.ones(N - 1); k_negative_rates = np.ones(N - 1)
# p_positive_rates = np.ones(N - 1); p_negative_rates = np.ones(N - 1)

# print(k_positive_rates)

x_tot_value = 5
y_tot_value = 4
x_tot = sp.Float(x_tot_value); y_tot = sp.Float(y_tot_value)

a = sp.Matrix(N, 1, lambda i,j: a_syms[i])

ones_vec = sp.ones(1, N - 1)

kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N - 1)])
kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N - 1)])
pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(1, N)])
pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(1, N)])

Kp = sp.diag(*kplus, 0)
# Kp = sp.diag(*[sp.Float(i) for i in k_positive_rates], sp.Float(0))

Km = sp.diag(*[sp.Float(i) for i in k_negative_rates]); Km = Km.row_insert(len(k_negative_rates), sp.zeros(1, len(k_negative_rates)))
# Km_small = sp.diag(*[sp.Float(i) for i in k_negative_rates])
# Km = Km_small.row_insert(len(k_negative_rates), sp.zeros(1, len(k_negative_rates)))  # becomes N x (N-1)

Pp = sp.diag(*pplus, 0)
# Pp = sp.diag(*[sp.Float(i) for i in p_positive_rates], sp.Float(0))

Pm = sp.diag(*[sp.Float(i) for i in p_negative_rates]); Pm = Pm.row_insert(len(p_negative_rates), sp.zeros(1, len(p_negative_rates)))

# Pm_small = sp.diag(*[sp.Float(i) for i in p_negative_rates])
# Pm = Pm_small.row_insert(len(p_negative_rates), sp.zeros(1, len(p_negative_rates)))  # becomes N x (N-1)

valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n)

Alpha_adjusted = np.delete(alpha_matrix, -1, axis = 0)
Beta_adjusted  = np.delete(beta_matrix, 0, axis = 0)

ones = sp.Matrix([1]*N)
Alpha_row_sums = alpha_matrix * ones
Beta_row_sums = beta_matrix * ones
DAlpha = sp.diag(*[Alpha_row_sums[i, 0] for i in range(N - 1)])
DBeta  = sp.diag(*[Beta_row_sums[i, 0] for i in range(N - 1)])
U = sp.diag(*[sp.Float(i) for i in k_negative_rates])
I = sp.diag(*[sp.Float(i) for i in p_negative_rates])
Q = sp.diag(*[sp.Float(i) for i in k_positive_rates]); Q = Q.col_insert(len(k_positive_rates), sp.zeros(len(k_positive_rates), 1))
D = sp.diag(*[sp.Float(i) for i in p_positive_rates]); D = D.col_insert(0, sp.zeros(len(p_positive_rates), 1))

M = U + DAlpha
N = I + DBeta
G = Km + Alpha_adjusted.T
H = Pm + Beta_adjusted.T
M_inv = M.inv(); N_inv = N.inv()

L1 = G * M_inv * Q - Kp; L2 = H * N_inv * D - Pp
W1 = M_inv * Q; W2 = N_inv * D
a_dot = x_tot * L1 * (1 / (1 + (ones_vec * W1 * a)[0])) * a + y_tot * L2 * (1 / (1 + (ones_vec * W2 * a)[0])) * a
polynomials = sp.simplify(a_dot * (1 + (ones_vec * W1 * a)[0]) * (1 + (ones_vec * W2 * a)[0]))
polynomials
# from sympy import solve_triangulated
# solve_triangulated(polynomials, *a_syms)

# 'lex' or 'grevlex' or 'grlex'
groebner_basis = sp.groebner(polynomials, *a_syms, order = 'lex')
groebner_basis
# roots = sp.solve(groebner_basis, *a_syms, dict = True)
# roots

NameError: name 'k_negative_rates' is not defined

# Computing adjusted matrices from piece of paper:

In [ ]:
from numpy import linalg as LA
import numpy as np
# from julia import Julia
# jl = Julia(compiled_modules=False)
# from julia import Main
import sympy as sp
from itertools import product
from scipy.optimize import root
from scipy.optimize import least_squares
# Initialize HomotopyContinuation in Julia
# Main.eval("using HomotopyContinuation")
# from julia import Julia
# jl = Julia(compiled_modules=False)
# from julia import Main
import numpy as np

# Load HomotopyContinuation once
# Main.eval("using HomotopyContinuation")
import numpy as np


def polynomial_finder(n, a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates):

    N = 2**n
    
    x_tot = sp.Float(x_tot_value); y_tot = sp.Float(y_tot_value); a_tot = sp.Float(a_tot_value)

    ones_vec = np.ones(N - 1)
    # ones_vec = np.ones((N-1, 1))
    Kp = np.diag(np.append(k_positive_rates, 0))
    Km = np.append(np.diag(k_negative_rates), np.zeros((1, len(k_negative_rates))), axis=0)

    Pp = np.diag(np.insert(p_positive_rates, 0, 0))
    Pm = np.vstack([np.zeros((1, len(p_negative_rates))), np.diag(p_negative_rates)])

    adjusted_alpha_mat = np.delete(alpha_matrix, -1, axis = 0)
    adjusted_beta_mat = np.delete(beta_matrix, 0, axis = 0)


    Da = np.diag(alpha_matrix[:-1, 1:] @ ones_vec)
    Db = np.diag(beta_matrix[1:, :-1] @ ones_vec)

    U = np.diag(k_negative_rates)
    I = np.diag(p_negative_rates)
    Q = Kp[:-1, :]
    D = np.delete(Pp, 0, axis=0)

    M = U + Da
    N = I + Db
    G = Km + adjusted_alpha_mat.T
    H = Pm + adjusted_beta_mat.T
    M_inv = np.linalg.inv(M); N_inv = np.linalg.inv(N)

    L1 = G @ M_inv @ Q - Kp; L2 = H @ N_inv @ D - Pp
    W1 = M_inv @ Q; W2 = N_inv @ D

    ####### RESCALING #######
    # k_neg_0 = k_negative_rates[0]
    # # t --> t * k_neg_0 * y_tot / a_tot
    # L1 = L1 * k_neg_0 / a_tot; L2 = L2 * k_neg_0 / a_tot
    # W1 = W1 / a_tot; W2 = W2 / a_tot

    return L1, L2, W1, W2
    
# def stable_fp_checker(a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates, verbose: bool = True): 
#     #Use n to determine the number of variables n = 2 # or pass n as an argument if needed N = 2 ** n 
#     # N = len(k_positive_rates) + 1
#     # print(N)
#     n = 2
#     N = 2**n
#     a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)], real = False) 
#     a = sp.Matrix(N, 1, lambda i,j: a_syms[i]) 

#     L1, L2, W1, W2 = polynomial_finder(n, a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates) 

#     ones_vec_sym = sp.Matrix([[1]* (N-1)])

#     L1_sym = sp.Matrix(np.array(L1, dtype=float).tolist())
#     L2_sym = sp.Matrix(np.array(L2, dtype=float).tolist())
#     W1_sym = sp.Matrix(np.array(W1, dtype=float).tolist())
#     W2_sym = sp.Matrix(np.array(W2, dtype=float).tolist())

#     x_tot = sp.Float(x_tot_value); y_tot = sp.Float(y_tot_value) 

#     inner_W1 = (ones_vec_sym * W1_sym * a)[0, 0]
#     inner_W2 = (ones_vec_sym * W2_sym * a)[0, 0]
    
#     L1a = (L1_sym * a)
#     L2a = (L2_sym * a)

#     p = sp.Float(float(x_tot / y_tot))

#     poly_exprs = []
#     for i in range(N):
#         expr = sp.simplify(p * (1 + inner_W2) * L1a[i, 0] + (1 + inner_W1) * L2a[i, 0])
#         poly_exprs.append(sp.expand(expr))
#     # polynomials = sp.expand(p * ((1 + (ones_vec_sym * W2_sym * a)[0,0])) * L1_sym * a + ((1 + (ones_vec_sym * W1_sym * a)[0,0])) * L2_sym * a)
#     print(poly_exprs)
#     # poly_exprs = [sp.nsimplify(eq, rational=True) for eq in poly_exprs]

#     # scale_factor = max([abs(c) for p in poly_exprs for c in p.as_coefficients_dict().values()])
#     # polys_scaled = [sp.simplify(p / scale_factor) for p in poly_exprs]
#     # print(polys_scaled)
#     # a_syms_list = [*a_syms]
#     # polys_scaled.append(sum(a_syms_list) - a_tot_value)
#     # print(polys_scaled)
#     # groebner_basis1 = sp.groebner(poly_exprs, *a_syms, order = 'grevlex') 
#     # print("Grobner basis 1:", groebner_basis1)
#     # polys_reduced = [g for g in groebner_basis1]
#     # groebner_basis = sp.groebner(polys_scaled, *a_syms, order = 'grevlex', domain = 'RR') 
#     # print("Grobner basis:", groebner_basis)

#     # poly_exprs.append(sum(a_syms) - a_tot_value)

#     scaled_polys = []

#     ##### SCALING
#     for poly in poly_exprs:
#         coeffs = [abs(c) for c in poly.as_coefficients_dict().values()]
#         scale_factor = max(coeffs) if coeffs else 1
#         scaled_polys.append(poly / scale_factor)
#     #####

#     polynomials_lambidified = sp.lambdify(a_syms, scaled_polys) # print(polynomials_lambidified(*[5, 10, 15, 20])) root_finder(polynomials_lambidified, low = 1e-1, high = 100, n_per_dim = 20) # print(sp.latex(polynomials)) 
#     # print(polynomials_lambidified)
#     # roots = root_finder(polynomials_lambidified, low = 1e-1, high = 100, n_per_dim = 20)
#     # roots = sp.nsolve(scaled_polys, a_syms, [a_tot_value / N] * N, prec=15)
    
    
#     # try:
#     #     roots = sp.nsolve(poly_exprs, a_syms, [a_tot_value / N] * N)
#     # except ValueError:
#     #     print("nsolve failed for this parameter set")
#     #     return []  # or fill with NaNs
#     roots = np.ravel(np.array(roots).astype(np.float64))
#     print(roots)
#     # print("sum of roots: ", np.sum(roots)) 
#     # roots
#     root_array = np.empty((0, 4), dtype=float) 
#     stable_fp_array = [] 

#     conservation_check = lambda root_array: np.sum(root_array) 
#         # for i in range(len(root_array)): 
    
#     if np.all(roots > 0): 
#         print("Positivity confirmed") 
#         # continue 
#     if np.sum(roots) - a_tot_value <= 1e-5: 
#         print("Conservation confirmed") 
#         # continue 
#     if np.all(stability_calculator(roots, x_tot_value / y_tot_value, L1, L2, W1, W2)) < 0:
#         print("Fixed point is stable!")

#         stable_fp_array.append(root_array) 
    
#     else:
#         print("Fixed point is not stable!") 

    
#     # for i in range(len(roots)): 
#     #     if len(roots) != 4: 
#     #         break 
        
#     #     print([roots[i][a_syms[0]], roots[i][a_syms[1]], roots[i][a_syms[2]], roots[i][a_syms[3]]]) 
#         # print(roots[i]) 
#         # root_array = np.append(root_array, np.array([[roots[i][a_syms[0]], roots[i][a_syms[1]], roots[i][a_syms[2]]]]), axis = 0) 
#         # root_array = np.append(root_array, np.array([[roots[i][a_syms[0]], roots[i][a_syms[1]], roots[i][a_syms[2]], roots[i][a_syms[3]]]]), axis = 0) # print(root_array) 
#         # conservation_check = lambda root_array: np.sum(root_array) 
#         # for i in range(len(root_array)): 
#         #     if np.all(root_array[i] > 0): 
#         #         print("Positivity confirmed") 
#         #         continue 
#         #     if np.sum(root_array[i]) - 1 <= 1e-5: 
#         #         print("Conservation confirmed") 
#         #         continue 
#         #     if stability_calculator(root_array[i], x_tot_value / y_tot_value, L1, L2, W1, W2): 
#         #         print("Stability confirmed") 
#         #         stable_fp_array.append(root_array[i]) 
#     return stable_fp_array

from itertools import product

def stability_calculator(a_fixed_points, p, L1, L2, W1, W2):
    N = len(a_fixed_points)
    ones_vec_j = np.ones((1, N-1))  # shape (1, N-1)
    a_fixed_points = np.array(a_fixed_points).reshape((N, 1))  # shape (N, 1)

    L1 = np.array(L1, dtype=float)
    L2 = np.array(L2, dtype=float)
    W1 = np.array(W1, dtype=float)
    W2 = np.array(W2, dtype=float)

    # Compute denominators
    denom1 = 1 + float(ones_vec_j @ W1 @ a_fixed_points)
    denom2 = 1 + float(ones_vec_j @ W2 @ a_fixed_points)

    # Compute terms
    term1 = L1 / denom1 - ((L1 @ a_fixed_points @ ones_vec_j @ W1) / denom1**2)
    term2 = L2 / denom2 - ((L2 @ a_fixed_points @ ones_vec_j @ W2) / denom2**2)
    J = p * term1 - term2

    eigenvalues = LA.eigvals(J)
    real_parts = np.real(eigenvalues)
    # print(real_parts)
    return real_parts
    
# def root_finder(polynomials_lambidified, low, high, n_per_dim):
#     unique_roots = []
#     ranges = [(low, high)] * 4
#     grids = [np.linspace(low, high, n_per_dim) for (low, high) in ranges]

#     for idx, (x0_0, x0_1, x0_2, x0_3) in enumerate(product(*grids)):
#         x0 = np.array([x0_0, x0_1, x0_2, x0_3], dtype=float)
#         roots = root(lambda arr: np.array(polynomials_lambidified(*arr)).flatten(), x0)
#         if (roots > 0).all():
#             unique_roots.append(roots.x)
    
#     return unique_roots

def stable_fp_checker(a_tot_value, x_tot_value, y_tot_value,
                      alpha_matrix, beta_matrix,
                      k_positive_rates, k_negative_rates,
                      p_positive_rates, p_negative_rates):
    n = 2
    N = 2**n

    # Unknowns: a0, a1, ..., a_{N-2} (last one is from conservation)
    a_syms = sp.symbols([f"a_{i}" for i in range(N-1)], real=True)
    
    # Compute L1, L2, W1, W2
    L1, L2, W1, W2 = polynomial_finder(
        n, a_tot_value, x_tot_value, y_tot_value,
        alpha_matrix, beta_matrix,
        k_positive_rates, k_negative_rates,
        p_positive_rates, p_negative_rates
    )
    
    L1 = np.array(L1, dtype=float)
    L2 = np.array(L2, dtype=float)
    W1 = np.array(W1, dtype=float)
    W2 = np.array(W2, dtype=float)

    # Build polynomials in terms of N-1 unknowns
    poly_exprs = []
    ones_vec = np.ones((1, N-1))
    
    for i in range(N-1):
        # a_N-1 in terms of conservation
        a_N = a_tot_value - sum(a_syms)
        a_full = list(a_syms) + [a_N]

        # Compute inner products for denominator
        denom1 = 1 + ones_vec @ W1 @ np.array(a_full).reshape((N,1))
        denom2 = 1 + ones_vec @ W2 @ np.array(a_full).reshape((N,1))

        # Jacobian-like terms
        term1 = L1 @ np.array(a_full).reshape((N,1)) / denom1 - \
                (L1 @ np.array(a_full).reshape((N,1)) @ ones_vec @ W1) / denom1**2
        term2 = L2 @ np.array(a_full).reshape((N,1)) / denom2 - \
                (L2 @ np.array(a_full).reshape((N,1)) @ ones_vec @ W2) / denom2**2

        p_ratio = x_tot_value / y_tot_value
        J_i = p_ratio * term1 - term2
        poly_exprs.append(J_i[i,0])

    # Lambdify
    poly_lambd = sp.lambdify(a_syms, poly_exprs, "numpy")

    # Initial guess
    # x0_arrays = [np.ones(N-1) * i * (a_tot_value / N) for i in range(10)]
    x0 = np.ones(N-1) * (a_tot_value / N)

    # Solve
    sol = root(lambda x: np.array(poly_lambd(*x), dtype=float).flatten(), x0)
    if not sol.success:
        return []  # failed

    # Recover full solution including a_N
    a_solution = list(sol.x) + [a_tot_value - sum(sol.x)]
    print(a_solution)
    print(np.sum(a_solution))
    # Check positivity
    # if :
    #     print("Positivity confirmed")

    # Compute stability
    eigenvalues = stability_calculator(a_solution, x_tot_value / y_tot_value, L1, L2, W1, W2)
    if np.all(np.array(a_solution) > 0) and np.all(eigenvalues < 0) and np.sum(a_solution) == 200:
        print("Positivity confirmed, conservation confirmed, and stability confirmed!")
        return [a_solution]
    else:
        print("Fixed point is not stable")
        return []
    
def log_uniform_sample(a: float, b: float, n: int, base: float = np.e) -> np.ndarray:
    """
    Sample n points uniformly in logarithmic space from range (a, b).

    Args:
        a (float): Lower bound (must be > 0).
        b (float): Upper bound (must be > a).
        n (int): Number of samples to draw.
        base (float): Logarithmic base (default: natural log).

    Returns:
        np.ndarray: Array of shape (n,) with log-uniformly sampled values.
    """
    if a <= 0 or b <= 0:
        raise ValueError("a and b must be positive.")
    if a >= b:
        raise ValueError("a must be smaller than b.")

    log_a = np.log(a) / np.log(base)
    log_b = np.log(b) / np.log(base)
    log_samples = np.random.uniform(log_a, log_b, n)
    samples = base ** log_samples
    return samples

def simulation(simulation_size):
    a_tot_value = 200
    x_tot_value_parameter_array = np.array([log_uniform_sample(1e-4, 1e-1, n=1, base=np.e) for _ in range(simulation_size)])
    y_tot_value_parameter_array = np.array([log_uniform_sample(1e-4, 1e-1, n=1, base=np.e) for _ in range(simulation_size)])
    alpha_matrix_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=4, base=np.e) for _ in range(simulation_size)])
    beta_matrix_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=4, base=np.e) for _ in range(simulation_size)])
    k_positive_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    k_negative_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    p_positive_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])
    p_negative_parameter_array = np.array([log_uniform_sample(1e-1, 1e3, n=3, base=np.e) for _ in range(simulation_size)])

    final_array = np.empty((0, 27), dtype=float) 

    for i in range(simulation_size):
    
        x_tot_value = x_tot_value_parameter_array[i][0]
        y_tot_value = y_tot_value_parameter_array[i][0]

        alpha_matrix = np.array([
            [0, alpha_matrix_parameter_array[i][0], alpha_matrix_parameter_array[i][1], 0],
            [0, 0, 0, alpha_matrix_parameter_array[i][2]],
            [0, 0, 0, alpha_matrix_parameter_array[i][3]],
            [0, 0, 0, 0]
        ])

        beta_matrix = np.array([
            [0, 0, 0, 0],
            [beta_matrix_parameter_array[i][0], 0, 0, 0],
            [beta_matrix_parameter_array[i][1], 0, 0, 0],
            [0, beta_matrix_parameter_array[i][2], beta_matrix_parameter_array[i][3], 0]
        ])

        k_positive_rates = k_positive_parameter_array[i]
        k_negative_rates = k_negative_parameter_array[i]
        p_positive_rates = p_positive_parameter_array[i]
        p_negative_rates = p_negative_parameter_array[i]

        stable_fp_array = stable_fp_checker(a_tot_value, x_tot_value, y_tot_value, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
        print(stable_fp_array)
        # Flatten stable_fp_array to 1D if it is not empty, else fill with NaNs
        if len(stable_fp_array) > 0:
            stable_fp_flat = np.ravel(stable_fp_array)
            array = np.concatenate([stable_fp_flat, 
                                   np.array([a_tot_value]), 
                                   np.array([x_tot_value]), 
                                   np.array([y_tot_value]),
                                   np.ravel(alpha_matrix_parameter_array[i]), 
                                   np.ravel(beta_matrix_parameter_array[i]), 
                                   np.ravel(k_positive_parameter_array[i]), 
                                   np.ravel(k_negative_parameter_array[i]), 
                                   np.ravel(p_positive_parameter_array[i]), 
                                   np.ravel(p_negative_parameter_array[i])])
            final_array = np.append(final_array, [array], axis = 0)

    header_string = "a_0_stable_fp, a_1_stable_fp, a_2_stable_fp, a_3_stable_fp, a_tot, x_tot, y_tot, alpha_matrix_parameters, beta_matrix_parameters, k_+_parameters, k_-_parameters, p_+_parameters, p_-_parameters"
    filename = f"test_{simulation_size}.csv"
    np.savetxt(filename, final_array, delimiter=",", header=header_string, comments='')
    return 

simulation(1000)


[np.float64(8.553995164304498), np.float64(165.1087855635109), np.float64(-28.74406654765595), np.float64(55.08128581984056)]
200.0
[-1.56165564e+00 -9.15617396e-01  1.27271970e-17  4.75051464e-01]
Fixed point is not stable
[]
[]
[np.float64(0.3081516629521406), np.float64(-0.5576032811345999), np.float64(-1.3879815770780926), np.float64(201.63743319526054)]
200.0
[-5.78002017e+01 -5.36872432e-01 -3.41562973e-01 -2.76817520e-16]
Fixed point is not stable
[]
[]
[np.float64(201.66635017433157), np.float64(-0.02208910141743673), np.float64(-1.6457084166413947), np.float64(0.0014473437272499723)]
200.0
[3.75316227e+01 1.75549038e-01 5.48000037e-18 2.90736910e-02]
Fixed point is not stable
[]
[np.float64(-10.83313652194129), np.float64(213.35090518527417), np.float64(-0.15434137204217327), np.float64(-2.363427291290691)]
200.0
[ 4.57580155e-02  4.57580155e-02 -8.90275023e-02 -3.07956384e-18]
Fixed point is not stable
[]
[np.float64(0.5106117166116328), np.float64(-108.28860516177468), np.fl

/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_54226/3974938788.py:197: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  denom1 = 1 + float(ones_vec_j @ W1 @ a_fixed_points)
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_54226/3974938788.py:198: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  denom2 = 1 + float(ones_vec_j @ W2 @ a_fixed_points)


[np.float64(0.6709475329026946), np.float64(-1.638646226101274), np.float64(-0.00011759595609721502), np.float64(200.96781628915468)]
200.0
[-1.64444298e+04 -5.98434779e+01 -1.87743612e+01 -2.08117777e-16]
Fixed point is not stable
[]
[]
[]
[]
[np.float64(243.64585693368582), np.float64(-0.06394671675230278), np.float64(0.3051346532989177), np.float64(-43.887044870232444)]
200.0
[ 1.41098322e+02 -1.34570215e+01 -3.14158885e-16 -2.21654508e-03]
Fixed point is not stable
[]
[]
[np.float64(-14.81352119704638), np.float64(0.6413370143042803), np.float64(219.46396015123858), np.float64(-5.291775968496495)]
200.0
[ 1.20729908e+01 -6.85515200e-02  1.60088657e-15 -3.66179803e-01]
Fixed point is not stable
[]
[]
[]
[]
[]
[]
[np.float64(161.71511172374153), np.float64(48.69451367275275), np.float64(-5.919189734618274), np.float64(-4.490435661876006)]
200.0
[4.90559743e-01 4.90559743e-01 1.78366425e-16 2.28326077e-01]
Fixed point is not stable
[]
[]
[]
[np.float64(-6.382931786368959), np.float64(

In [136]:
import numpy as np
from julia import Julia
jl = Julia(compiled_modules=False)
from julia import Main

# Load HomotopyContinuation in Julia once
Main.eval("using HomotopyContinuation")

def log_uniform_sample(a: float, b: float, n: int, base: float = np.e) -> np.ndarray:
    """Sample n points uniformly in log space from range (a, b)."""
    log_a = np.log(a)/np.log(base)
    log_b = np.log(b)/np.log(base)
    samples = base ** np.random.uniform(log_a, log_b, n)
    return samples

def simulation(simulation_size: int):
    a_tot_value = 25
    
    # Generate parameters with reasonable ranges
    x_tot_array = log_uniform_sample(1e-3, 1e-1, simulation_size)
    y_tot_array = log_uniform_sample(1e-3, 1e-1, simulation_size)
    alpha_array = log_uniform_sample(0.1, 1e3, (simulation_size, 4))
    beta_array  = log_uniform_sample(0.1, 1e3, (simulation_size, 4))
    k_plus_array = log_uniform_sample(0.1, 1e3, (simulation_size, 3))
    k_minus_array = log_uniform_sample(0.1, 1e3, (simulation_size, 3))
    p_plus_array = log_uniform_sample(0.1, 1e3, (simulation_size, 3))
    p_minus_array = log_uniform_sample(0.1, 1e3, (simulation_size, 3))

    final_array = []

    for i in range(simulation_size):
        x_tot_value = x_tot_array[i]
        y_tot_value = y_tot_array[i]

        # Construct alpha and beta matrices
        alpha_matrix = np.array([
            [0, alpha_array[i][0], alpha_array[i][1], 0],
            [0, 0, 0, alpha_array[i][2]],
            [0, 0, 0, alpha_array[i][3]],
            [0, 0, 0, 0]
        ])
        beta_matrix = np.array([
            [0, 0, 0, 0],
            [beta_array[i][0], 0, 0, 0],
            [beta_array[i][1], 0, 0, 0],
            [0, beta_array[i][2], beta_array[i][3], 0]
        ])

        k_positive_rates = k_plus_array[i]
        k_negative_rates = k_minus_array[i]
        p_positive_rates = p_plus_array[i]
        p_negative_rates = p_minus_array[i]

        # Use the new robust HomotopyContinuation checker
        stable_fp_array = stable_fp_checker_hc(
            a_tot_value, x_tot_value, y_tot_value,
            alpha_matrix, beta_matrix,
            k_positive_rates, k_negative_rates,
            p_positive_rates, p_negative_rates
        )

        # Flatten or fill with NaNs if no positive fixed point
        if len(stable_fp_array) > 0:
            stable_fp_flat = np.ravel(stable_fp_array)
        else:
            stable_fp_flat = np.full(4, np.nan)

        array = np.concatenate([
            stable_fp_flat,
            np.ravel(alpha_array[i]),
            np.ravel(beta_array[i]),
            np.ravel(k_positive_rates),
            np.ravel(k_negative_rates),
            np.ravel(p_positive_rates),
            np.ravel(p_negative_rates)
        ])
        final_array.append(array)

    final_array = np.array(final_array)
    header_string = "a_0,a_1,a_2,a_3," + \
                    "alpha0,alpha1,alpha2,alpha3," + \
                    "beta0,beta1,beta2,beta3," + \
                    "k_plus0,k_plus1,k_plus2," + \
                    "k_minus0,k_minus1,k_minus2," + \
                    "p_plus0,p_plus1,p_plus2," + \
                    "p_minus0,p_minus1,p_minus2"

    np.savetxt(f"simulation_{simulation_size}.csv", final_array, delimiter=",", header=header_string, comments='')

# Example run
simulation(10)


[]
[]
[]


KeyboardInterrupt: 

In [266]:
groebner_basis = sp.groebner(polynomials, *a_syms)
groebner_basis

GroebnerBasis([1.0*a_0, 1.0*a_1], a_0, a_1, domain='RR', order='lex')

In [264]:
roots = sp.solve(groebner_basis, *a_syms, dict = True, simplify=False)
roots

[{a_0: 0.0, a_1: 0.0}]

In [69]:
import numpy as np

# Initialize an empty 2D array with 0 rows and 3 columns
empty_array = np.empty((0, 3), dtype=int) 
print(empty_array)
# Create the array to append
new_row = np.array([[10, 20, 30]])

# Append the new row
empty_array = np.append(empty_array, new_row, axis=0)

# Append another row
another_row = np.array([[40, 50, 60]])
empty_array = np.append(empty_array, another_row, axis=0)

print(empty_array)

for i in range(len(empty_array)):
    print(empty_array[i])
    if np.all(empty_array[i] > 0):
        print("Positivity confirmed")

[]
[[10 20 30]
 [40 50 60]]
[10 20 30]
Positivity confirmed
[40 50 60]
Positivity confirmed


In [40]:
import numpy as np

def sample_log_uniform(n, a, b, size=1, rng=None):
    """
    Sample points uniformly in log-space from an n-dimensional hypercube.

    Parameters
    ----------
    n : int
        Number of dimensions (hypercube dimensionality).
    a, b : float or array-like of length n
        Lower and upper bounds for each coordinate. Must be > 0.
        If scalar, the same range (a,b) is used for all dimensions.
    size : int
        Number of samples to draw (returns array of shape (size, n)).
    rng : None or np.random.Generator
        Optional numpy random Generator for reproducible draws.

    Returns
    -------
    samples : ndarray, shape (size, n)
        Samples drawn so that log(samples) is uniformly distributed
        in the n-dimensional box [log(a), log(b)]^n.
    """
    if rng is None:
        rng = np.random.default_rng()

    a = np.asarray(a)
    b = np.asarray(b)

    # allow scalar bounds or per-dimension bounds
    if a.shape == () and b.shape == ():
        log_a = np.log(a)
        log_b = np.log(b)
        low = log_a
        high = log_b
    else:
        a = a.reshape(-1)
        b = b.reshape(-1)
        if a.size != n or b.size != n:
            raise ValueError("If a or b are arrays they must have length n.")
        low = np.log(a)
        high = np.log(b)

    if np.any(high <= low):
        raise ValueError("Require b > a (elementwise) and both > 0.")

    # draw uniformly in log-space then exponentiate
    u = rng.uniform(low=low, high=high, size=(size, n))
    return np.exp(u)

# Example 3: reproducible draws
rng = np.random.default_rng(42)
samples = sample_log_uniform(n=3, a=10e-1, b=10e7, size=10, rng=None)
print(samples)

[[5.78304517e+01 1.82956144e+02 2.71619238e+07]
 [2.04273050e+00 1.16511733e+06 1.36519063e+05]
 [1.68285696e+06 2.83459746e+03 3.62775105e+06]
 [6.67450880e+02 9.40138456e+03 5.84308078e+01]
 [1.21057522e+00 1.74155583e+01 5.89683717e+03]
 [2.57308998e+05 1.95258514e+05 2.50466426e+01]
 [2.30474971e+01 3.41773373e+05 1.25218651e+06]
 [5.55052535e+04 1.92117067e+02 3.31439272e+07]
 [8.00624336e+01 2.00289200e+02 5.03063927e+01]
 [3.78813091e+06 7.92873229e+02 1.01604200e+05]]
